<a href="https://colab.research.google.com/github/zgbl/WeiqiBoardDetect/blob/main/notebooks/BoardDetect3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
import io

In [2]:
class GoBoardAnalyzer:
    def __init__(self):
        self.board_size = 19  # 标准围棋棋盘19x19
        self.corners = []
        self.grid_points = []
        self.stones = []

In [ ]:
    def upload_and_load_image(self):
        """上传并加载图像"""
        print("请上传围棋棋盘图像...")
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]

        # 读取图像
        image_data = uploaded[filename]
        image = Image.open(io.BytesIO(image_data))

        # 转换为OpenCV格式
        self.original_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        self.image = self.original_image.copy()

        print(f"图像加载成功，尺寸: {self.image.shape}")
        return self.image

In [ ]:
    def detect_board_corners(self):
        """检测棋盘四角"""
        # 转换为灰度图
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

        # 高斯模糊
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # 边缘检测
        edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

        # 霍夫直线检测
        lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)

        if lines is not None:
            # 分离水平和垂直线
            horizontal_lines = []
            vertical_lines = []

            for line in lines:
                rho, theta = line[0]
                if abs(theta) < 0.1 or abs(theta - np.pi) < 0.1:  # 水平线
                    horizontal_lines.append((rho, theta))
                elif abs(theta - np.pi/2) < 0.1:  # 垂直线
                    vertical_lines.append((rho, theta))

            # 找到边界线
            if horizontal_lines and vertical_lines:
                h_rhos = [rho for rho, theta in horizontal_lines]
                v_rhos = [rho for rho, theta in vertical_lines]

                # 棋盘边界
                top_rho = min(h_rhos)
                bottom_rho = max(h_rhos)
                left_rho = min(v_rhos)
                right_rho = max(v_rhos)

                # 计算四个角点
                height, width = self.image.shape[:2]

                # 简化的角点计算
                self.corners = [
                    (int(left_rho), int(abs(top_rho))),      # 左上
                    (int(right_rho), int(abs(top_rho))),     # 右上
                    (int(left_rho), int(abs(bottom_rho))),   # 左下
                    (int(right_rho), int(abs(bottom_rho)))   # 右下
                ]

        print(f"检测到棋盘角点: {self.corners}")
        return self.corners

In [ ]:
    def detect_grid_lines(self):
        """检测棋盘网格线和交叉点"""
        if len(self.corners) < 4:
            print("需要先检测棋盘角点")
            return []

        # 使用角点信息计算网格
        top_left, top_right, bottom_left, bottom_right = self.corners

        # 计算棋盘区域
        x_min = min(top_left[0], bottom_left[0])
        x_max = max(top_right[0], bottom_right[0])
        y_min = min(top_left[1], top_right[1])
        y_max = max(bottom_left[1], bottom_right[1])

        # 生成网格交叉点
        self.grid_points = []
        for i in range(self.board_size):
            for j in range(self.board_size):
                x = x_min + (x_max - x_min) * j / (self.board_size - 1)
                y = y_min + (y_max - y_min) * i / (self.board_size - 1)
                self.grid_points.append((int(x), int(y)))

        print(f"生成了 {len(self.grid_points)} 个网格交叉点")
        return self.grid_points

In [ ]:
    def detect_stones(self):
        """检测黑白棋子及其位置"""
        if not self.grid_points:
            print("需要先检测网格点")
            return []

        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

        # 使用HoughCircles检测圆形（棋子）
        circles = cv2.HoughCircles(
            gray,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=20,
            param1=50,
            param2=30,
            minRadius=8,
            maxRadius=25
        )

        self.stones = []
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")

            for (x, y, r) in circles:
                # 找到最近的网格点
                min_dist = float('inf')
                closest_grid_idx = -1

                for idx, (gx, gy) in enumerate(self.grid_points):
                    dist = np.sqrt((x - gx)**2 + (y - gy)**2)
                    if dist < min_dist and dist < 25:  # 阈值距离
                        min_dist = dist
                        closest_grid_idx = idx

                if closest_grid_idx != -1:
                    # 判断黑白子
                    # 提取棋子区域的平均亮度
                    mask = np.zeros(gray.shape, dtype=np.uint8)
                    cv2.circle(mask, (x, y), r-2, 255, -1)
                    mean_intensity = cv2.mean(gray, mask=mask)[0]

                    # 根据亮度判断黑白
                    stone_color = 'white' if mean_intensity > 127 else 'black'

                    # 转换为棋盘坐标
                    grid_row = closest_grid_idx // self.board_size
                    grid_col = closest_grid_idx % self.board_size

                    self.stones.append({
                        'position': (grid_row, grid_col),
                        'pixel_pos': (x, y),
                        'color': stone_color,
                        'radius': r
                    })

        print(f"检测到 {len(self.stones)} 个棋子")
        return self.stones


In [ ]:
    def visualize_results(self):
        """可视化检测结果"""
        result_image = self.original_image.copy()

        # 绘制角点
        for i, corner in enumerate(self.corners):
            cv2.circle(result_image, corner, 8, (0, 255, 0), -1)
            cv2.putText(result_image, f'C{i+1}',
                       (corner[0]+10, corner[1]-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 绘制网格点
        for point in self.grid_points:
            cv2.circle(result_image, point, 2, (255, 0, 0), -1)

        # 绘制棋子
        for stone in self.stones:
            x, y = stone['pixel_pos']
            color = (0, 0, 255) if stone['color'] == 'black' else (255, 255, 255)
            cv2.circle(result_image, (x, y), stone['radius'], color, 2)

            # 添加位置标注
            pos_text = f"{stone['position']}"
            cv2.putText(result_image, pos_text, (x-20, y-stone['radius']-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)

        # 显示结果
        plt.figure(figsize=(12, 8))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB))
        plt.title('原始图像')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        plt.title('检测结果')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

In [ ]:
    def analyze_board_state(self):
        """分析棋盘状态"""
        print("\n=== 棋盘分析结果 ===")
        print(f"检测到角点数量: {len(self.corners)}")
        print(f"网格交叉点数量: {len(self.grid_points)}")
        print(f"棋子总数: {len(self.stones)}")

        # 统计黑白子
        black_count = sum(1 for stone in self.stones if stone['color'] == 'black')
        white_count = sum(1 for stone in self.stones if stone['color'] == 'white')

        print(f"黑子数量: {black_count}")
        print(f"白子数量: {white_count}")

        print("\n棋子位置详情:")
        for stone in self.stones:
            print(f"位置 {stone['position']}: {stone['color']} 子")

In [ ]:
    def run_full_analysis(self):
        """运行完整分析流程"""
        print("开始围棋棋盘分析...")

        # 1. 上传并加载图像
        self.upload_and_load_image()

        # 2. 检测棋盘角点
        self.detect_board_corners()

        # 3. 检测网格线
        self.detect_grid_lines()

        # 4. 检测棋子
        self.detect_stones()

        # 5. 可视化结果
        self.visualize_results()

        # 6. 分析棋盘状态
        self.analyze_board_state()

        return {
            'corners': self.corners,
            'grid_points': self.grid_points,
            'stones': self.stones
        }

In [ ]:
# 使用示例
def main():
    # 创建分析器实例
    analyzer = GoBoardAnalyzer()

    # 运行完整分析
    results = analyzer.run_full_analysis()

    return analyzer, results

In [ ]:
if __name__ == "__main__":
    print("围棋棋盘分析器已准备就绪！")
    print("运行 main() 函数开始分析")

    # 可以直接运行
    # analyzer, results = main()